In [1]:
import breeze.linalg.Matrix.castOps
import breeze.linalg.Vector.castOps
import org.apache.spark.SparkConf
import org.apache.spark.sql.functions.{array_contains, avg, col, concat, concat_ws, countDistinct, element_at, explode, explode_outer, lit, lower, mean, round, split, substring, sum, udf, upper, when}
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.{Column, DataFrame, SaveMode, SparkSession, functions}
import shapeless.Lazy.apply
import java.net.URL
import java.net.HttpURLConnection
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import org.apache.spark.sql.SQLContext
import org.apache.spark.input.PortableDataStream
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import java.io.{ IOException, FileOutputStream, FileInputStream, File }
import java.util.zip.{ ZipEntry, ZipInputStream }
import com.amazonaws.services.s3.AmazonS3Client
import com.amazonaws.auth.BasicAWSCredentials
//import com.datastax.spark.connector.cql.CassandraConnector
//import org.apache.spark.sql.cassandra._

// Imports
import scala.sys.process._
import java.net.URL
import java.io.File
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import java.net.HttpURLConnection 
import org.apache.spark.sql.functions._

//import scala.sqlContext.implicits._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions.{col, to_date, to_timestamp}
import org.apache.spark.sql.types.DateType
import org.apache.spark.sql.functions.{from_unixtime, unix_timestamp, _}

import org.apache.spark.input.PortableDataStream
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import org.apache.spark.sql.SQLContext
import com.amazonaws.services.s3.AmazonS3Client
import com.amazonaws.auth.BasicAWSCredentials
import org.apache.spark.sql.types.IntegerType
import com.amazonaws.services.s3.{AmazonS3, AmazonS3ClientBuilder}

val conf = new SparkConf().setAll(Map(
    "spark.master" -> "local",
    "spark.scheduler.mode" -> "FIFO",
    "spark.speculation" -> "false",
    "spark.reducer.maxSizeInFlight" -> "48m",
    "spark.serializer" -> "org.apache.spark.serializer.KryoSerializer",
    "spark.kryoserializer.buffer.max" -> "1g",
    "spark.shuffle.file.buffer" -> "32k",
    "spark.default.parallelism" -> "12",
    "spark.sql.shuffle.partitions" -> "12"
))

val spark = SparkSession.builder.config(conf).appName("SparKGDELT").getOrCreate()

import spark.implicits._

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1611184029712_0007,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import breeze.linalg.Matrix.castOps
import breeze.linalg.Vector.castOps
import org.apache.spark.SparkConf
import org.apache.spark.sql.functions.{array_contains, avg, col, concat, concat_ws, countDistinct, element_at, explode, explode_outer, lit, lower, mean, round, split, substring, sum, udf, upper, when}
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.{Column, DataFrame, SaveMode, SparkSession, functions}
import shapeless.Lazy.apply
import java.net.URL
import java.net.HttpURLConnection
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import org.apache.spark.sql.SQLContext
import org.apache.spark.input.PortableDataStream
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import java.io.{IOException, FileOutputStream, FileInputStream, File}
import java.util.zip.{ZipEntry, ZipInputStream}
import com.amazonaws.services.s3.AmazonS3Client
import com.amazonaws.auth.BasicAWS

In [8]:
// English Data
val list_csv = spark.read.format("csv").option("delimiter", " ").
    csv("s3://testfuret/masterfilelist_translation.txt").
    withColumnRenamed("_c0","size").
    withColumnRenamed("_c1","hash").
    withColumnRenamed("_c2","url")

val list_2020_tot = list_csv.where(col("url").like("%/2020020%")) 
//val list_2020_tot = list_csv.where(col("url").like("%export%")) 
list_2020_tot.show(10,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

list_csv: org.apache.spark.sql.DataFrame = [size: string, hash: string ... 1 more field]
list_2020_tot: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [size: string, hash: string ... 1 more field]
+-------+--------------------------------+--------------------------------------------------------------------------------+
|size   |hash                            |url                                                                             |
+-------+--------------------------------+--------------------------------------------------------------------------------+
|60089  |4fd11d42d646dd8ed194981cc2fd9ee6|http://data.gdeltproject.org/gdeltv2/20200201000000.translation.export.CSV.zip  |
|55386  |1a5818ddada7124e1ee34724871ac67b|http://data.gdeltproject.org/gdeltv2/20200201000000.translation.mentions.CSV.zip|
|6178636|72d0e5ca10362beca19a0510854f4721|http://data.gdeltproject.org/gdeltv2/20200201000000.translation.gkg.csv.zip     |
|62076  |6352fe6b69a145d656a1464bcf035b5f|http://

In [2]:
//Ouverture des fichiers ZIP et écriture en csv.

val gkgRDD = sc.binaryFiles("s3://testfuret/Master_file /*2020030*.gkg.csv.zip",100).    
   flatMap {  
       
       // decompresser les fichiers
       case (name: String, content: PortableDataStream) =>

          val zis = new ZipInputStream(content.open)
          Stream.continually(zis.getNextEntry).
                takeWhile(_ != null).
                //takeWhile{case null => zis.close(); false
                          //case _ => true}.
                flatMap { _ =>
                    val br = new BufferedReader(new InputStreamReader(zis))
                    Stream.continually(br.readLine()).takeWhile(_ != null)
                }
    }

val gkgDF = gkgRDD.map(x => x.split("\t")).toDF()//.map(row => row.mkString(";")).map(x => x.split(";")).toDF()
//val gkgDF = gkgRDD.map(x => x.split("\t")).map(row => row.mkString(";")).toDF()

val dfGKG = gkgDF.withColumn("_tmp", $"value").select(
    $"_tmp".getItem(0).as("GKGRECORDID"),
    $"_tmp".getItem(1).as("V2DATE"),
    $"_tmp".getItem(3).as("V2SOURCECOMMONNAME"),
    $"_tmp".getItem(4).as("V2DOCUMENTIDENTIFIER"),
    $"_tmp".getItem(7).as("V1THEMES"),
    $"_tmp".getItem(9).as("V1LOCATIONS"),
    $"_tmp".getItem(11).as("V1PERSONS"),
    $"_tmp".getItem(15).as("V1TONE")
    )



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gkgRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[1] at flatMap at <console>:83
gkgDF: org.apache.spark.sql.DataFrame = [value: array<string>]
dfGKG: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 6 more fields]


In [3]:
//Ouverture des fichiers ZIP et écriture en csv.
val mentionENRDD = sc.binaryFiles("s3://testfuret/Master_file /*2020030*.mentions*",100).    
   flatMap {  
       
       // decompresser les fichiers
       case (name: String, content: PortableDataStream) =>

          val zis = new ZipInputStream(content.open)
          Stream.continually(zis.getNextEntry).
                takeWhile(_ != null).
                //takeWhile{case null => zis.close(); false
                          //case _ => true}.
                flatMap { _ =>
                    val br = new BufferedReader(new InputStreamReader(zis))
                    Stream.continually(br.readLine()).takeWhile(_ != null)
                }

    }

val mentionENDF = mentionENRDD.map(x => x.split("\t")).map(row => row.mkString(";")).map(x => x.split(";")).toDF()

val dfMentionEN = mentionENDF.withColumn("_tmp", $"value").select(
     $"_tmp".getItem(0).as("globaleventID"),
     $"_tmp".getItem(4).as("MentionSourceName"),
     $"_tmp".getItem(5).as("MentionIdentifier"),
     $"_tmp".getItem(14).as("MentionDocTranslationInfo")
    )



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mentionENRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[4] at flatMap at <console>:82
mentionENDF: org.apache.spark.sql.DataFrame = [value: array<string>]
dfMentionEN: org.apache.spark.sql.DataFrame = [globaleventID: string, MentionSourceName: string ... 2 more fields]


In [4]:
def cleanLangue(langue: String): String = {
	if (langue == null)
    	"eng" 
	else
        langue.substring(6)
}

val cleanLangueUdf = udf(cleanLangue _)

val dfMention: DataFrame = dfMentionEN.
	withColumn("Langue", cleanLangueUdf($"MentionDocTranslationInfo")).
    drop($"MentionDocTranslationInfo")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cleanLangue: (langue: String)String
cleanLangueUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
dfMention: org.apache.spark.sql.DataFrame = [globaleventID: string, MentionSourceName: string ... 2 more fields]


In [5]:
//Ouverture des fichiers ZIP et écriture en csv.

val EventENRDD = sc.binaryFiles("s3://testfuret/Master_file /2020030*export.CSV.zip",100).    
   flatMap {  
       
       // decompresser les fichiers
       case (name: String, content: PortableDataStream) =>

          val zis = new ZipInputStream(content.open)
          Stream.continually(zis.getNextEntry).
                takeWhile(_ != null).
                //takeWhile{case null => zis.close(); false
                          //case _ => true}.
                flatMap { _ =>
                    val br = new BufferedReader(new InputStreamReader(zis))
                    Stream.continually(br.readLine()).takeWhile(_ != null)
                }

    }

val eventENDF = EventENRDD.map(x => x.split("\t")).map(row => row.mkString(";")).map(x => x.split(";")).toDF()

val dfEvent = eventENDF.withColumn("_tmp", $"value").select(
     $"_tmp".getItem(0).as("globaleventID"),
     $"_tmp".getItem(1).as("day"),
     $"_tmp".getItem(2).as("month"),
     $"_tmp".getItem(3).as("year"),
     $"_tmp".getItem(31).as("NumMentions"),
     $"_tmp".getItem(32).as("NumSources"),
     $"_tmp".getItem(33).as("NumArticles"),
     $"_tmp".getItem(34).as("AvgTone"),
     $"_tmp".getItem(52).as("ActionGeo_Fullname")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

EventENRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[9] at flatMap at <console>:83
eventENDF: org.apache.spark.sql.DataFrame = [value: array<string>]
dfEvent: org.apache.spark.sql.DataFrame = [globaleventID: string, day: string ... 7 more fields]


In [6]:
val dfGKG_Pand = dfGKG.
    withColumn("CheckCOVID", $"V1THEMES".like("%PANDEMIC%") || $"V1THEMES".like("%CORONAVIRUS%"))

val dfGKG_Covid = dfGKG_Pand.
    filter((dfGKG_Pand("CheckCOVID") === "true"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGKG_Pand: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
dfGKG_Covid: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GKGRECORDID: string, V2DATE: string ... 7 more fields]


In [7]:
val dfGKGCovidWithMention = dfGKG_Covid.join(dfMention, dfGKG_Covid("V2DOCUMENTIDENTIFIER")===dfMention("MentionIdentifier"), "inner")

//dfGKGCovidWithMention.write.parquet("s3://axel-michalewicz-telecom/GKGMention/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGKGCovidWithMention: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 11 more fields]


In [8]:
//val dfGKGMention = spark.
      //read.
      //parquet("s3://axel-michalewicz-telecom/GKGMention/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
val dfFinal = dfGKGCovidWithMention.join(dfEvent, dfGKGCovidWithMention("globaleventID")===dfEvent("globaleventID"), "inner").drop(dfEvent("globaleventID"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfFinal: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 19 more fields]


In [9]:
dfFinal.write.json("s3://cgl2/json/mars_01/")   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Job aborted.
  at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
  at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:174)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.

In [45]:
//--------------------------------------------------------------------->

val dfCovid_Pays = dfFinal.
      withColumn("Pays", split($"ActionGeo_Fullname", ", ")).
      withColumn("Pays", element_at($"Pays", -1))

// 1°) afficher le nombre d’articles/évènements qui parlent de COVID qu’il y a eu pour chaque triplet (jour, pays de l’évènement, langue de l’article).

val CountEventArticle = dfCovid_Pays.
	groupBy("day", "Pays", "Langue").
    agg(countDistinct("globaleventID").alias("NbEvent"),countDistinct("GKGRECORDID").alias("NbArticle"))

CountEventArticle.write.csv("s3://axel-michalewicz-telecom/requete1/janv10")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfCovid_Pays: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 20 more fields]
CountEventArticle: org.apache.spark.sql.DataFrame = [day: string, Pays: string ... 3 more fields]


In [ ]:
//--------------------------------------------------------------------->

// 2°) pour un pays donné en paramètre,
// affichez les évènements qui y ont eu place triées par le nombre de mentions (tri décroissant);
// permettez une agrégation par jour/mois/année

val dfCovid_Country = dfCovid_Pays.
	groupBy("Pays","day","globaleventID").
    agg(sum("NumMentions").alias("countMention")).
    sort($"countMention".desc)

//Si aggrégation par jour, Mois, Année, utiliser le champs "day"

//Requête  question 2)
//dfCovid_Country.show()

dfCovid_Country.write.csv("s3://axel-michalewicz-telecom/requete2/janv10")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
//--------------------------------------------------------------------->
// 3°) pour une source de données passée en paramètre (gkg.SourceCommonName)
// affichez les thèmes (V1THEMES de la table GKG), personnes (V1PERSONS), lieux (V1LOCATIONS plutôt que Actor1Geo_Fullname, Actor2Geo_Fullname, ActionGeo_Fullname) dont les articles de cette source parlent
// ainsi que le nombre d’articles (NumArticles dans la table Event ? => NON count juste le nombre de ligne de la table GKG car une ligne est un article) et le ton moyen des articles (AvgTone dans la table Event ? => .agg(mean(V1.TONE) ) (pour chaque thème/personne/lieu);
// permettez une agrégation par jour/mois/année (=day).

val dfGKGQuestion3 = dfGKG_Covid.
	withColumn("Thèmes", split($"V1THEMES", ";")).
    withColumn("Personnes", split($"V1PERSONS", ";")).
    withColumn("Tons", split($"V1TONE", ",")).
    withColumn("Lieux", split($"V1LOCATIONS", ";"))
    
val dfCovid_1DataSource = dfGKGQuestion3.
    withColumn("Annee", substring($"V2DATE", 0,4)).
    withColumn("Mois", substring($"V2DATE", 0,6)).
    withColumn("Jour", substring($"V2DATE", 0,8)).
    select($"Annee",$"Mois",$"Jour",$"V2SOURCECOMMONNAME",$"GKGRECORDID",$"Thèmes", $"Personnes", $"Lieux", $"Tons".getItem(0).alias("AVGTon"))


//Requête  question 3): thèmes
val dfCovid_1DataSourceTheme = dfCovid_1DataSource.
	select($"V2SOURCECOMMONNAME",$"GKGRECORDID", $"Annee", $"Mois", $"Jour",$"AVGTon", explode_outer($"Thèmes").alias("Thème"))

val dfCovid_1DataSourceThemeGroup = dfCovid_1DataSourceTheme.
	groupBy("V2SOURCECOMMONNAME","Jour","Thème").
    agg(countDistinct("GKGRECORDID").alias("NbArticles"), mean("AVGTon").alias("TonMoyen"))

dfCovid_1DataSourceThemeGroup.write.csv("s3://axel-michalewicz-telecom/requete3/Theme/janv10")

//dfCovid_1DataSourceThemeGroup.show()

//Requête  question 3): personnes
val dfCovid_1DataSourcePersonnes = dfCovid_1DataSource.
	select($"V2SOURCECOMMONNAME",$"GKGRECORDID", $"Annee", $"Mois", $"Jour",$"AVGTon", explode_outer($"Personnes").alias("Personne"))

val dfCovid_1DataSourcePersonnesGroup = dfCovid_1DataSourcePersonnes.
	groupBy("V2SOURCECOMMONNAME","Jour","Personne").
    agg(countDistinct("GKGRECORDID").alias("NbArticles"), mean("AVGTon").alias("TonMoyen"))

//dfCovid_1DataSourcePersonnesGroup.show()

dfCovid_1DataSourcePersonnesGroup.write.csv("s3://axel-michalewicz-telecom/requete3/Personnes/janv10")


//Requête  question 3): Lieux
val dfCovid_1DataSourceLieux = dfCovid_1DataSource.
	select($"V2SOURCECOMMONNAME",$"GKGRECORDID", $"Annee", $"Mois", $"Jour",$"AVGTon", explode_outer($"Lieux").alias("Lieu"))


val dfCovid_1DataSourceLieuxGroup = dfCovid_1DataSourceLieux.
	groupBy("V2SOURCECOMMONNAME","Jour","Lieu").
    agg(countDistinct("GKGRECORDID").alias("NbArticles"), mean("AVGTon").alias("TonMoyen"))

val dfCovid_1DataSourceLieuxGroupFinal = dfCovid_1DataSourceLieuxGroup.
    withColumn("locat", split($"Lieu", "#")).
    select($"V2SOURCECOMMONNAME",$"Jour",$"locat".getItem(1), $"NbArticles", $"TonMoyen")

//dfCovid_1DataSourceLieuxGroupFinal.show()

dfCovid_1DataSourceLieuxGroupFinal.write.csv("s3://axel-michalewicz-telecom/requete3/Lieux/janv10")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGKGQuestion3: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 11 more fields]
dfCovid_1DataSource: org.apache.spark.sql.DataFrame = [Annee: string, Mois: string ... 7 more fields]
dfCovid_1DataSourceTheme: org.apache.spark.sql.DataFrame = [V2SOURCECOMMONNAME: string, GKGRECORDID: string ... 5 more fields]
dfCovid_1DataSourceThemeGroup: org.apache.spark.sql.DataFrame = [V2SOURCECOMMONNAME: string, Jour: string ... 3 more fields]
dfCovid_1DataSourcePersonnes: org.apache.spark.sql.DataFrame = [V2SOURCECOMMONNAME: string, GKGRECORDID: string ... 5 more fields]
dfCovid_1DataSourcePersonnesGroup: org.apache.spark.sql.DataFrame = [V2SOURCECOMMONNAME: string, Jour: string ... 3 more fields]
dfCovid_1DataSourceLieux: org.apache.spark.sql.DataFrame = [V2SOURCECOMMONNAME: string, GKGRECORDID: string ... 5 more fields]
dfCovid_1DataSourceLieuxGroup: org.apache.spark.sql.DataFrame = [V2SOURCECOMMONNAME: string, Jour: string ... 3 more fields]
dfCovid_1DataSourceLieuxGrou